# Тема: "Парсинг данных. HTML, DOM, XPath"

### Задание 1

Написать приложение и функцию, которые собирают основные новости с сайта на выбор **dzen.ru**, **lenta.ru**, **mail.ru** . Для парсинга использовать **XPath**.

Структура данных должна содержать:

* название источника;

* наименование новости;

* ссылку на новость;

* дата публикации.

Выполнить минимум один сайт, максимум все.


#### Ответ:

In [1]:
import requests
from lxml import html
from pprint import pprint
from datetime import datetime

In [2]:
URL_DZENRU = 'https://dzen.ru/news'
URL_MAILRU = 'https://news.mail.ru'
URL_LENTARU = 'https://lenta.ru'

HEADER = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) \
          AppleWebKit/537.36 (KHTML, like Gecko) Chrome/104.0.0.0 Safari/537.36'}

#### Парсинг новостей Dzen

In [3]:
params = {'sso_failed': '',}

def get_content_dom(url, headers=None, params=None):
    response = requests.get(url, headers = HEADER, params = params)
    content_dom = html.fromstring(response.text)

    print(response.url)
    
    return content_dom

In [4]:
def parser_dzen_news(content_dom):
    
    news_container = content_dom.xpath("//div[contains(@class,'mg-card ')]")
    
    dzen_news = []
    for new_container in news_container:
        element = dict()
        
        news_name = new_container.xpath(".//h2/a/text()")
        element['name'] = str(news_name[0]).replace(u'\xa0', u' ')
        
        news_link = new_container.xpath(".//h2/a/@href")
        element['link'] = str(news_link[0])
        
        news_date = new_container.xpath(".//span[@class='mg-card-source__time']/text()")
        element['date'] = str(news_date[0]).strip()
        
        news_source = new_container.xpath(".//a[@class='mg-card__source-link']/text()")
        element['source'] = str(news_source[0])
        
        dzen_news.append(element) 

    return dzen_news

#### Парсинг новостей Mail

In [5]:
def parser_mail_news(content_dom):
    
    news_container = content_dom.xpath('//ul[contains(@class, "list")]/li[contains(@class, "list__item")]')
    
    mail_news = []
    for item in news_container:
        element = dict()

        news_name = item.xpath('.//span/a[contains(@class, "list__text")]/text()')
        element['name'] = str(news_name[0]).replace(u'\xa0', u' ')

        news_link = item.xpath('.//span/a/@href')
        element['link'] = str(news_link[0])
        
        response = requests.get(element['link'], headers = HEADER)
        dom = html.fromstring(response.text)

        news_date = dom.xpath('.//span/@datetime')
        news_date = datetime.strptime(str(news_date[0]), '%Y-%m-%dT%H:%M:%S%z')
        element['date'] = news_date.strftime('%H:%M')

        news_source = dom.xpath('//span[contains(@class, "note")]/*/span[contains(@class, "link__text")]/text()')
        element['source'] = str(news_source[0])

        mail_news.append(element)

    return mail_news

#### Парсинг новостей Lenta

In [6]:
def parser_lenta_news(content_dom):
    
    news_container = content_dom.xpath('//div[contains(@class, "main-container")]')
    
    lenta_news = []
    for item in news_container:
        element = dict()

        news_name = item.xpath('.//span[contains(@class, "card-mini__title")]/text()')
        element['name'] = str(news_name[0]).replace(u'\xa0', u' ')

        news_link = item.xpath('.//a[contains(@class, card-mini__topnews)]/@href')
        element['link'] = str(URL_LENTARU) + str(news_link[0])

        news_date = item.xpath('.//time[contains(@class, "card-mini__date")]/text()')
        element['date'] = str(news_date[0]).strip()

        element['source'] = 'lenta.ru'

        lenta_news.append(element)

    return lenta_news

In [7]:
def get_news(content_dom, parser_name = 'dzen'):
    result = None
    
    if parser_name == 'dzen':
      result = parser_dzen_news(content_dom)
    elif parser_name == 'mail':
      result = parser_mail_news(content_dom)
    elif parser_name == 'lenta':
      result = parser_lenta_news(content_dom)
        
    return result

In [8]:
dom = get_content_dom(URL_DZENRU, headers = HEADER, params = params)

https://dzen.ru/news?issue_tld=ru&sso_failed=


In [9]:
dzen_news = get_news(dom, 'dzen')
dzen_news

[{'name': 'Советник главы Крыма Крючков объяснил взрыв в районе Армянска тренировкой военных',
  'link': 'https://dzen.ru/news/story/Sovetnik_glavy_Kryma_Kryuchkov_obyasnil_vzryv_vrajone_Armyanska_trenirovkoj_voennykh--60dc062e729e132fa3620d6aef190d27?lang=ru&rubric=index&fan=1&stid=0GmTW78W3s8IjTJv0sHb&t=1676553016&tt=true&persistent_id=2008567537&story=7a3b423b-a557-5eb9-b1a0-f806d4201297&issue_tld=ru',
  'date': '23:07',
  'source': 'РИА Новости'},
 {'name': 'Госдума приняла закон о госязыке, ограничивающий использование иностранных слов',
  'link': 'https://dzen.ru/news/story/Gosduma_prinyala_zakon_ogosyazyke_ogranichivayushhij_ispolzovanie_inostrannykh_slov--cd73c1c8e84a6676aced6a03eb341d2e?lang=ru&rubric=index&fan=1&stid=jXql6VcWOSv6aL6vqlo_&t=1676553016&tt=true&persistent_id=2008561278&story=4379d556-f873-5da2-8ee1-bc926fc8b0c6&issue_tld=ru',
  'date': '23:05',
  'source': 'РИА Новости'},
 {'name': 'Лукашенко заявил о готовности воевать против Украины в случае агрессии против Бе

In [10]:
dom, response = get_content_dom(URL_MAILRU, headers = HEADER)

https://news.mail.ru/


In [11]:
mail_news = get_news(dom, 'mail')
mail_news

[{'name': 'Лукашенко назвал условие, при котором будет воевать против Киева',
  'link': 'https://news.mail.ru/politics/55085391/',
  'date': '12:42',
  'source': '© РИА Новости'},
 {'name': 'Фото: первые снимки затонувшего «Титаника» после крушения',
  'link': 'https://news.mail.ru/society/55089153/',
  'date': '15:52',
  'source': 'Новости Mail.ru'},
 {'name': 'На Гавайях проснулся вулкан Килауэа',
  'link': 'https://pogoda.mail.ru/news/55087473/',
  'date': '15:57',
  'source': 'Вокруг Света'},
 {'name': 'Новые услуги для пенсионеров: за что теперь не надо платить',
  'link': 'https://news.mail.ru/society/55079896/',
  'date': '10:23',
  'source': 'Life.ru'},
 {'name': 'Wildberries запретил продавцам фейковые скидки',
  'link': 'https://news.mail.ru/economics/55088652/',
  'date': '15:03',
  'source': 'Коммерсантъ'},
 {'name': 'Какие новые штрафы ждут владельцев домов и дач в 2023 году',
  'link': 'https://news.mail.ru/economics/55079897/',
  'date': '10:28',
  'source': 'Life.ru'},


In [12]:
dom, response = get_content_dom(URL_LENTARU, headers = HEADER)

https://lenta.ru/


In [13]:
lenta_news = get_news(dom, 'lenta')
lenta_news

[{'name': 'Переодевавшийся в женщину россиянин похитил у пенсионерок более миллиона рублей',
  'link': 'https://lenta.ru/news/2023/02/16/chvk/',
  'date': '16:18',
  'source': 'lenta.ru'},
 {'name': 'Омбудсмен Львова-Белова попросила у Путина «немножечко денег»',
  'link': 'https://lenta.ru/articles/2023/02/16/menopause/',
  'date': '15:54',
  'source': 'lenta.ru'},
 {'name': 'Футболист клуба РПЛ рассказал об участии в странных матчах',
  'link': 'https://lenta.ru/news/2023/02/16/strannouchastie/',
  'date': '15:36',
  'source': 'lenta.ru'},
 {'name': 'Беременная Рианна назвала первые дни материнства «выносом мозга»',
  'link': 'https://lenta.ru/news/2023/02/16/asap/',
  'date': '15:18',
  'source': 'lenta.ru'}]

---

END